In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import sys
import scipy
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud

%matplotlib inline
sns.set_style('darkgrid')

from IPython.display import display

In [3]:
train = pd.read_csv('../common/data/train.csv', index_col='id')
test = pd.read_csv('../common/data/test.csv', index_col='id')

FileNotFoundError: File b'../common/data/train.csv' does not exist

In [ ]:
ratings = train.loc[:, 'toxic':'identity_hate']
train['is_toxic'] = train.loc[:, 'toxic':].any(axis=1)
train['n_toxic'] = ratings.sum(axis=1).astype(int)

In [ ]:
raw_comments = train['comment_text']

In [ ]:
default_params = dict(
    ngram_range=(1,2),
    stop_words='english',
    max_features=1000,
    binary=True # on hypothesis that this will be effective at detecting obscenity
)



In [ ]:
# Skree plot
def skree_plot(data=pca_data, n_components=10):
    pca = PCA(n_components=n_components)
    pca.fit(data)

    chartdata = pca.explained_variance_ratio_ 

    fig, ax1 = plt.subplots()
    ax1.plot(pd.Series(chartdata))
    ax1.set_xlabel('Principal Component')
    ax1.set_ylabel('Variance Explained')

    ax2 = ax1.twinx()
    ax2.plot(pd.Series(chartdata).cumsum())
    ax2.set_ylabel('Cumulative Explained')

    fig.tight_layout()
    plt.show()

In [ ]:
def plot_pca(
    component=1,
    data=pca_data,
    n_features=5):
    
    pca = PCA(n_components=component)
    pca.fit(data)
    
    idx = np.argsort(np.abs(pca.components_[component - 1]))[:-(n_features + 1):-1]
    weights = pca.components_[component - 1][idx]
    words = data.columns[idx]
    ax = pd.Series(weights).plot(kind='bar')
    ax.set_xticklabels(words)
    [tick.set_rotation(0) for tick in ax.get_xticklabels()];
    return ax   

In [ ]:
# Vectorize text
toxic_comments = raw_comments#[train['is_toxic']]

toxic_vect = vectorize_text(
    toxic_comments, 
    kind='count',
    output='df',
    param_dict=dict(
        ngram_range=(1,1),
        stop_words='english',
        max_features=100,
        binary=False
    )
)

display(skree_plot(data=toxic_vect, n_components=10))

In [ ]:
pca = PCA(n_components=7)
pca.fit(toxic_vect)
toxic_components = pd.DataFrame(
    pca.transform(toxic_vect), 
    index=toxic_comments.index)

In [ ]:
plot_pca(data=toxic_vect, component=1, n_features=5)

In [ ]:
def get_poles_idx(data=toxic_components, component=1, percentile=0.5, pole='high'):
    if pole == 'high':
        return data.index[data.iloc[:, component - 1] < np.percentile(data.iloc[:, component - 1], q=percentile)]
    elif pole == 'low':
        return data.index[data.iloc[:, component - 1] < np.percentile(data.iloc[:, component - 1], q=1 - percentile)]
    
train.loc[get_poles_idx(toxic_components, component=1, percentile=0.95, pole='high')]

In [ ]:
cloud = WordCloud()
high_pc1 = 